# Угадай число от 1 до 100
## Первый git-проект в SkillFactory

Игра заключается в следующем: рандомно "загадывается" число от 1 до 100, необходимо его угадать с помощью скрипта за наименьшее число попыток.

In [1]:
import numpy as np

In [27]:
'''Игра вручную'''

count = 0                            # счетчик попыток
number = np.random.randint(1,101)    # загадали число
print ("Загадано число от 1 до 99")

while True:                        # бесконечный цикл
    predict = int(input())         # предполагаемое число
    count += 1                     # плюсуем попытку
    
    if number == predict: break    # выход из цикла, если угадали
    elif number > predict: print (f"Угадываемое число больше {predict} ")
    elif number < predict: print (f"Угадываемое число меньше {predict} ")
        
print (f"Вы угадали число {number} за {count} попыток.")

Загадано число от 1 до 99
56
Угадываемое число меньше 56 
45
Угадываемое число меньше 45 
13
Угадываемое число больше 13 
25
Угадываемое число больше 25 
35
Угадываемое число меньше 35 
30
Вы угадали число 30 за 6 попыток.


In [47]:
'''Исходный алгоритм, предложенный SkillFactory - перебор всех значений от 1 по порядку. Алгоритм запускается 1000 раз, чтобы вычислить среднее число попыток.'''

def game_core_v1(number):
    '''
    Просто угадываем перебором никак не используя информацию о больше или меньше.
    Функция принимает загаданное число и возвращает число попыток.
    '''
    count = 0
    
    while True:
        count+=1
        predict = np.random.randint(1,101) # предполагаемое число
        
        if number == count: 
            return(count) # выход из цикла, если угадали  


def score_game(game_core_v1):
    '''Запускаем игру 1000 раз, чтоб узнать как быстро игра угадывает число'''
    count_ls = []
    np.random.seed(1)  # фиксируем RANDOM SEED, чтобы ваш эксперимент был воспроизводим!
    random_array = np.random.randint(1, 101, size=(1000))       #поправлено мной на интервал (1, 100, size=(1000)) для избежания бесконечного цикла
    
    for number in random_array:
        count_ls.append(game_core_v1(number))
    
    score = int(np.mean(count_ls))
    
    print(f"Ваш алгоритм угадывает число в среднем за {score} попыток")
    
    return(score)

In [48]:
score_game(game_core_v1)     # checking the average number of tries

Ваш алгоритм угадывает число в среднем за 50 попыток


50

In [49]:
'''Модификация функции поиска - также предложена SkillFatory.'''

def game_core_v2(number):
    '''
    Сначала устанавливаем любое random число, а потом уменьшаем или увеличиваем его в зависимости от того,
    больше оно или меньше нужного. Функция принимает загаданное число и возвращает число попыток.
    '''
    
    count = 0
    predict = np.random.randint(1,101)
    
    while number != predict:
        count+=1
        
        if number > predict: 
            predict += 1
        elif number < predict: 
            predict -= 1
    
    return(count) # выход из цикла, если угадали

In [50]:
score_game(game_core_v2)     # checking the average number of tries

Ваш алгоритм угадывает число в среднем за 32 попыток


32

In [51]:
def game_core_v3(number):
    '''
    My first own method - realization of the same logic as when playing manually. 
    The predicted number is compared to the random one, the interval of search gets 
    reduced depending on the result of comparison operation.
    '''
    
    count = 0
    low = 1
    high = 101
    
    while True:    # infinite cycle
        predict = np.random.randint(low, high)     # [low; high) - 'low' inclusively, 'high' exclusively
        count += 1
        
        if number == predict:
            break
        elif number > predict:
            low = predict + 1    # set predicted number as lower threshold
        elif number < predict:
            high = predict       # set predicted number as higher threshold

    return count

In [52]:
score_game(game_core_v3)     # checking the average number of tries

Ваш алгоритм угадывает число в среднем за 7 попыток


7

In [53]:
def game_core_v4(number):
    '''
    The second method - dividing intervals in halves.
    '''
    
    count = 0
    low = 1
    high = 101
    
    while True:
        predict = low + (high - low) // 2
        count += 1
        
        if number == predict:
            break
        elif number > predict:
            low = predict + 1    # set predicted number as lower threshold
        elif number < predict:
            high = predict       # set predicted number as higher threshold

    return count

In [54]:
score_game(game_core_v4)     # checking the average number of tries

Ваш алгоритм угадывает число в среднем за 5 попыток


5

#### Попробуем совместить разработанные нами методы и сравним быстродействие:

In [55]:
def game_core_v5(number):
    '''
    The third method - a combination of first two methods.
    '''
    
    count = 0
    low = 1
    high = 101
    
    while True:
        predict1 = np.random.randint(low, high)     # this methods tries two numbers at once
        predict2 = low + (high - low) // 2
        count += 1
        
        maximum = max(predict1, predict2)     # comparing two predicted numbers, setting them as min and max for defining an interval if the number is in range
        minimum = min(predict1, predict2)
        
        if number == predict1:
            break
        
        elif number == predict2:
            break
        
        elif number < minimum:
            high = minimum       # set as higher threshold
        
        elif number > maximum:
            low = maximum + 1    # sets as lower threshold
        
        elif number in range(minimum, maximum):     # checking if the number is in between
            count += 1
            low = minimum + 1
            high = maximum

    return count

In [56]:
score_game(game_core_v5)

Ваш алгоритм угадывает число в среднем за 5 попыток


5

Как видим, быстродействие не повысилось.

## Проверяем, в каких интервалах какой из методов наиболее эффективен

Изменяем функции, чтобы передавать верхний и нижний пределы только один раз, сравниваем быстродействие трех методов. Цель - определить, будет ли какой-то из методов более эффективен в более узком/широком интервале, чтобы применять каждый из методов в соответствующих интервалах (по мере сокращения основного).

In [14]:
def compare_methods():
    """
    Comparison of the speed of the three developed methods.
    """
    
    low = int(input())
    high = int(input())
    
    score_game1(game_core_v21, low, high)
    score_game1(game_core_v31, low, high)
    score_game1(game_core_v41, low, high)
    
    return True


def score_game1(game_core_v1, low, high):
    '''
    Запускаем игру 1000 раз, чтоб узнать как быстро игра угадывает число.
    '''
    
    count_ls = []
    np.random.seed(1)  # фиксируем RANDOM SEED, чтобы ваш эксперимент был воспроизводим!
    
    random_array = np.random.randint(low, high, size=(1000))       #поправлено мной на интервал (1, 100, size=(1000)) для избежания бесконечного цикла
    
    for number in random_array:
        count_ls.append(game_core_v1(number, low, high))
    
    score = int(np.mean(count_ls))
    
    print(f"Ваш алгоритм угадывает число в среднем за {score} попыток")
    
    return(score)


def game_core_v21(number, low, high):
    '''
    Сначала устанавливаем любое random число, а потом уменьшаем или увеличиваем его в зависимости от того,
    больше оно или меньше нужного. Функция принимает загаданное число и возвращает число попыток.
    Метод предложен Skillfactory.
    '''
    
    count = 0
    predict = np.random.randint(low,high)
    
    while number != predict:
        count+=1
        
        if number > predict: 
            predict += 1
        elif number < predict: 
            predict -= 1
    
    return(count) # выход из цикла, если угадали


def game_core_v31(number, low, high):
    '''
    My first own method - realization of the same logic as when playing manually. 
    The predicted number is compared to the random one, the interval of search gets 
    reduced depending on the result of comparison operation.
    '''
    
    count = 0
    
    while True:    # infinite cycle
        predict = np.random.randint(low, high)     # [low; high) - 'low' inclusively, 'high' exclusively
        count += 1
        
        if number == predict:
            break
        elif number > predict:
            low = predict + 1    # set as lower threshold
        elif number < predict:
            high = predict       # set as higher threshold

    return count


def game_core_v41(number, low, high):
    '''
    The second method - dividing intervals in halves.
    '''
    
    count = 0
    
    while True:
        predict = low + (high - low) // 2
        count += 1
        
        if number == predict:
            break
        elif number > predict:
            low = predict + 1    # set as lower threshold
        elif number < predict:
            high = predict       # set as higher threshold

    return count

In [15]:
compare_methods()

1
101
Ваш алгоритм угадывает число в среднем за 32 попыток
Ваш алгоритм угадывает число в среднем за 7 попыток
Ваш алгоритм угадывает число в среднем за 5 попыток


True

In [16]:
compare_methods()

1
51
Ваш алгоритм угадывает число в среднем за 16 попыток
Ваш алгоритм угадывает число в среднем за 6 попыток
Ваш алгоритм угадывает число в среднем за 4 попыток


True

In [17]:
compare_methods()

1
26
Ваш алгоритм угадывает число в среднем за 8 попыток
Ваш алгоритм угадывает число в среднем за 4 попыток
Ваш алгоритм угадывает число в среднем за 3 попыток


True

In [18]:
compare_methods()

1
13
Ваш алгоритм угадывает число в среднем за 4 попыток
Ваш алгоритм угадывает число в среднем за 3 попыток
Ваш алгоритм угадывает число в среднем за 3 попыток


True

In [23]:
compare_methods()

1
10
Ваш алгоритм угадывает число в среднем за 2 попыток
Ваш алгоритм угадывает число в среднем за 3 попыток
Ваш алгоритм угадывает число в среднем за 2 попыток


True

In [24]:
compare_methods()

1
8
Ваш алгоритм угадывает число в среднем за 2 попыток
Ваш алгоритм угадывает число в среднем за 2 попыток
Ваш алгоритм угадывает число в среднем за 2 попыток


True

In [25]:
compare_methods()

1
7
Ваш алгоритм угадывает число в среднем за 1 попыток
Ваш алгоритм угадывает число в среднем за 2 попыток
Ваш алгоритм угадывает число в среднем за 2 попыток


True

## Вывод

Видим, что метод деления интервала поиска пополам остается наиболее эффективным вплоть до сокращения интервала до \[1, 10), далее эффективность трех методов становится одинаковой (до [1, 8)), и в узком интервале [1, 7) первый метод, предложенный Skillfactory, быстрее всех находит нужное число. Напишем код, совмещающий первый и последний методы:

In [46]:
def game_core_v42(number, low, high):
    '''
    The second method - dividing intervals in halves.
    UPD: modified!
    '''
    
    count = 0
    
    while high-low != 6:
        predict = low + (high - low) // 2
        count += 1
        
        if number == predict:
            break
        elif number > predict:
            low = predict + 1    # set as lower threshold
        elif number < predict:
            high = predict       # set as higher threshold

    return [count, low, high]


def game_core_v22(number, low, high):
    '''
    Сначала устанавливаем любое random число, а потом уменьшаем или увеличиваем его в зависимости от того,
    больше оно или меньше нужного. Функция принимает загаданное число и возвращает число попыток.
    Метод предложен Skillfactory.
    UPD: modified!
    '''
    
    count = 0
    predict = np.random.randint(low,high)
    
    while number != predict:
        count+=1
        
        if number > predict: 
            predict += 1
        elif number < predict: 
            predict -= 1
    
    return count # выход из цикла, если угадали


def game_core_v6(number):
    """
    Combination of the two methods - division in halves and Skillfactory method 1.
    """
    
    low = 1
    high = 101
    
    result1 = game_core_v42(number, low, high) # results of executing the first method
    
    count = result1[0] # current number of tries
    low = result1[1]
    high = result1[2] # reassigning low and high according to the results of the first method
    
    count += game_core_v22(number, low, high) # adding the tries of using the second method
    
    return count

In [45]:
score_game(game_core_v6)

Ваш алгоритм угадывает число в среднем за 6 попыток


6

## Вывод:

Как видно, скорость угадывания числа от совмещения двух алгоритмов только снизилась, поэтому наиболее эффективным методом является деление интервала поиска пополам (game_core_v4).